# Lecture 8-2 : Interactive Data Visualization - bokeh

* Bokeh is an interactive visualization library that targets modern web browsers for presentation. Its goal is to provide elegant, concise construction of versatile graphics, and to extend this capability with high-performance interactivity over very large or streaming datasets. Bokeh can help anyone who would like to quickly and easily create interactive plots, dashboards, and data applications.
* To get started using Bokeh to make your visualizations, see the User Guide (https://docs.bokeh.org/en/latest/ ).

* To see examples of how you might use Bokeh with your own data, check out the Gallery 
(https://docs.bokeh.org/en/latest/docs/gallery.html ).

In [1]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource

In [2]:
import pandas as pd

df_covid = pd.read_csv('./covid19-worldwide2020.csv')

In [3]:
cases_and_death2 = df_covid.groupby('countriesAndTerritories')[['cases', 'deaths', 'popData2019', 'continentExp']].agg(
    {'cases':'sum', 'deaths':'sum', 'popData2019':'max', 'continentExp':'max'})

cases_and_death2 = cases_and_death2.reset_index()

colormap = {'Europe': 'red', 'Africa': 'orange', 'America': 'gray',
           'Asia': 'green', 'Oceania': 'blue'}

cases_and_death2['color'] = cases_and_death2['continentExp'].map(colormap)

data = ColumnDataSource(cases_and_death2)

x = cases_and_death2['cases']
y = cases_and_death2['deaths']

TOOLS="pan,wheel_zoom,zoom_in,zoom_out,reset,save"

tooltips = [
            ('country','@countriesAndTerritories'),
            ('cases', '@cases'),
            ('deaths', '@deaths')           
]

p = figure(plot_height=800,
           plot_width=1200,
           x_axis_label='cases',
           y_axis_label='deaths',
           title='relations between cases and deaths of Covid19 worldwide',
           tools=TOOLS)

for continent in list(df_covid['continentExp'].drop_duplicates()): 
   
    cases_and_death3 = cases_and_death2[cases_and_death2['continentExp']==continent]

    source = ColumnDataSource(
        cases_and_death3
    )
    p.circle(x='cases', y='deaths', source=source,
                         size=10, alpha=0.4, legend_group='continentExp', color='color',
                         hover_fill_color='black', hover_alpha=0.5)

p.add_tools(HoverTool(tooltips=tooltips))
p.legend.click_policy="hide" 
p.legend.location = "top_left"

output_file("cases-vs-deaths.html", title="Covid19 data visualization")
show(p)